class CNN_NET(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        x = nn.Softmax(dim=1)(x)
        return x## Directory Setup

In [ ]:
!mkdir data

In [ ]:
!cp ../input/dogs-vs-cats/* data

In [ ]:
cd data

In [ ]:
!unzip -q train.zip 

In [ ]:
!unzip -q test1.zip 

In [ ]:
cd ..

In [ ]:
ls data

# Config

In [ ]:
from torchvision import transforms 
import torch 
from glob import glob 
from pathlib import Path
import random 

class config:
    p = Path('.')
    images = list(p.glob('data/train/*.jpg'))
    random.shuffle(images)
    train_images, valid_images = images[:int(len(images) * 0.8)], images[int(len(images) * 0.8):]


    tfms = transforms.Compose([transforms.ToTensor()])

    INPUT_SIZE = 224*224*3
    NUM_CLASS = 2
    EPOCHS = 3
    BATCH_SIZE = 32
    LEARNING_RATE = 0.01
    device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Training Function

In [ ]:
import torch 
from tqdm import tqdm
from sklearn.metrics import accuracy_score


def train(model, train_dl, valid_dl, optimizer, loss_fn):
    model.train() 
    for epoch in range(config.EPOCHS):
        loop = tqdm(train_dl)
        train_losses = []
        train_acc = []
        val_losses = []
        val_accuracies = []
        for xb, yb in loop:
            xb = xb.to(config.device)
            yb = yb.to(config.device)
            out = model(xb)
            loss = loss_fn(out, yb)
            loss.backward()
            train_losses.append(loss.item())
            optimizer.step()
            optimizer.zero_grad()
            prediction = torch.argmax(out, dim=1)
            acc = accuracy_score(yb.cpu(), prediction.cpu())
            train_acc.append(acc)           
        model.eval()
        with torch.no_grad():
            for xb, yb in valid_dl:
                xb = xb.to(config.device)
                yb = yb.to(config.device)
                output = model(xb)
                loss = loss_fn(output, yb)
                prediction = torch.argmax(output, dim=1)
                accuracy = accuracy_score(yb.cpu(), prediction.cpu())
                val_losses.append(loss)
                val_accuracies.append(accuracy)
        print(f'epoch={epoch}, train_loss = {sum(train_losses)/len(train_losses)},\
            val_loss={sum(val_losses)/len(val_losses)}, train_acc = {sum(train_acc)/len(train_acc)}, \
                 val_acc={sum(val_accuracies)/len(val_accuracies)}')

In [ ]:
from torch.utils.data import Dataset, DataLoader
import os 
from glob import glob 
from torchvision import transforms, utils
import random 
from PIL import Image
from pathlib import Path 
import torch 




class DogsVsCatsDataset(Dataset):
    def __init__(self, images, transform=None):
        self.images = images
        self.transform = config.tfms
    
    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = self.images[idx]
        image = Image.open(img_path).resize((224,224))
        label = img_path.name.split('.')[0]
        label = 1 if label == 'cat' else 0
        if self.transform:
            image = self.transform(image)
        return image, torch.tensor(label)

if __name__ == '__main__':
    train_dataset = DogsVsCatsDataset(config.train_images)
    valid_dataset = DogsVsCatsDataset(config.valid_images)
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    valid_loader = DataLoader(valid_dataset, batch_size=32, shuffle=True)
    print(len(train_loader))
    print(len(valid_loader))

In [ ]:
import torch.nn as nn 
import torch.nn.functional as F
from torchvision import models

class Net(nn.Module):
    def __init__(self, input_size, num_class):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_size, 512)
        self.fc2 = nn.Linear(512, 124)
        self.fc3 = nn.Linear(124, 64)
        self.fc4 = nn.Linear(64, num_class)
    
    def forward(self, xb):
        xb = xb.view((xb.shape[0], -1))
        xb = F.relu(self.fc1(xb))
        xb = F.relu(self.fc2(xb))
        xb = F.relu(self.fc3(xb))
        xb = self.fc4(xb)
        out = nn.Softmax(dim=1)(xb)
        return xb

class CNN_NET(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.conv1 = nn.Conv2d(3, 64, 3)
        self.conv2 = nn.Conv2d(64, 128, 3)
        self.conv3 = nn.Conv2d(128, 256, 3)
        self.conv4 = nn.Conv2d(256, 512, 3)
        self.pool = nn.MaxPool2d(kernel_size=3)
        self.linear = nn.Linear(512, 10)
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = self.pool(F.relu(self.conv4(x)))
        x = self.linear(x.view(x.shape[0], -1))
        x = nn.Softmax(dim=1)(x)
        return x
    
class vgg(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.features = models.vgg16(pretrained=True)
        self.last_linear = nn.Linear(1000, 10)
    def forward(self, x):
        x = self.features(x)
        x = self.last_linear(x)
        x = nn.Softmax(dim=1)(x)
        return x

In [ ]:
xb, yb = next(iter(train_loader))
model = Net(input_size=config.INPUT_SIZE, num_class = config.NUM_CLASS)
# # model = CNN_NET()
# out = model(xb)
# print(out.shape)
loss_fn = nn.CrossEntropyLoss()

# loss = loss_fn(out, yb)
# loss
# for xb, yb in train_loader:
#     out = model(xb)
#     loss = loss_fn(out, yb)
#     print(loss)

In [ ]:
from torch.utils.data import DataLoader 
import torch 
import torch.nn as nn 
import numpy as np 
from torch.utils.data import Subset

if __name__=="__main__":
    train_dataset = DogsVsCatsDataset(config.train_images)
    valid_dataset = DogsVsCatsDataset(config.valid_images)
    train_dataset = Subset(train_dataset, np.arange(2044))
    valid_dataset = Subset(valid_dataset, np.arange(512))
    train_loader = DataLoader(train_dataset, batch_size=config.BATCH_SIZE, shuffle=True)
    valid_loader = DataLoader(valid_dataset, batch_size=config.BATCH_SIZE, shuffle=True)
#     model = Net(input_size=config.INPUT_SIZE, num_class = config.NUM_CLASS)
#     model = CNN_NET()
    model = vgg()
    model.to(config.device)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=config.LEARNING_RATE)
    device = config.device
    train(model, train_loader,valid_loader, optimizer=optimizer, loss_fn=loss_fn)